TO DO:
- lemmatize kws? 

In [1]:
import pandas as pd
import json

In [7]:
#read mji csv and turn into two column df for names and desc 
df_mji = pd.read_csv('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/DATA/MJI/MJI_data.csv', keep_default_na=False, dtype='string')
df_mji_small = df_mji.iloc[:, [0, 5]].copy()
df_mji_small['Title'] = df_mji_small['Title'].str.lower().str.strip()
df_mji_small['Description'] = df_mji_small['Description'].str.lower().str.strip()
df_mji_small

,Title,Description
0,marian anderson archive: the artist on the wor...,philadelphia born contralto singer marian ande...
1,the opera database,the opera database is a comprehensive database...
2,sfcm oral history project,the san francisco conservatory of music’s oral...
3,library of congress performing arts research g...,guides to library of congress holdings organiz...
4,yale’s oral history of american music,"founded in 1969, the oral history of american ..."
...,...,...
157,the pinewalk collection,over 200 recordings of dj sets from fire islan...
158,lynn abbott interviews,author and historian lynn abbott conducted int...
159,laurraine goreau interviews & archive,the interviews from the laurraine goreau colle...
160,sound archives of the institute for musicology,the im sound archives contain documents and re...


In [ ]:
#read musow json dump and turn into df w/ same columns 
with open('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/DATA/MUSOW/musow_name_desc_url_cat.json') as file:
    data = json.load(file)
    
musow_names = [result['name']['value'].strip().lower() for result in data['results']['bindings']]
musow_desc = [result['description']['value'].strip().lower() for result in data['results']['bindings']]
df_musow = pd.DataFrame(columns=['Title', 'Description'])
df_musow['Title'] = musow_names
df_musow['Description'] = musow_desc
df_musow = df_musow.astype('string')
df_musow

In [ ]:
#remove duplicates from MJI set
df_mji_small[~df_mji_small['Title'].isin(df_musow['Title'])].dropna()

In [15]:
#save each df to a text file for tokenizing 
with open('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/DATA/MJI/mji_corpus.txt', 'a') as f:
    dfAsString = df_mji_small.to_string(header=False, index=False)
    f.write(dfAsString)
with open('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/DATA/MUSOW/musow_corpus.txt', 'a') as f:
    dfAsString = df_musow.to_string(header=False, index=False)
    f.write(dfAsString)

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import PlaintextCorpusReader, wordnet

In [3]:
#set corpus cleaning variables 
mji_corpus = PlaintextCorpusReader('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/DATA/MJI/mji_corpus.txt', '.*\.txt')
musow_corpus = PlaintextCorpusReader('/Users/laurentfintoni/Desktop/University/COURSE DOCS/THESIS/Internship/DATA/MUSOW/musow_corpus.txt', '.*\.txt')
stopwords = nltk.corpus.stopwords.words('english')
punct_tokenizer = nltk.RegexpTokenizer(r"\w+")
custom_stopwords = ['available', '000', 'including', 'also', 'includes', 'website', 'new', 'include', 'well', 'based', 'source', 'sources', 'contains', 'search']

In [4]:
def keywords(input_corpus):
    string = input_corpus.raw('')
    tokenised = punct_tokenizer.tokenize(string)
    clean = [w for w in tokenised if w not in stopwords]
    clean_2 = [w for w in clean if w not in custom_stopwords]
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(w) for w in clean_2]
    freqdist_lem = nltk.FreqDist(lemmatized)
    most_common_lem = freqdist_lem.most_common(35)
    most_common_lem_list = []
    for t in most_common_lem:
        most_common_lem_list.append(t[0])
    bigrams = nltk.bigrams(lemmatized)
    freqdist_bg = nltk.FreqDist(bigrams)
    search_bigrams = []
    for k, v in freqdist_bg.items():
        if k[0] in most_common_lem_list:
            if k[1] in most_common_lem_list:
                if v > 5:
                    k = ' '.join(k)
                    search_bigrams.append([k, v])
    kw_pd = pd.DataFrame(columns=['Most Common KW', 'KW Freq', 'Bigrams', 'Bigram Freq'])
    kw_pd['Most Common KW'] = pd.Series([w[0] for w in most_common_lem])
    kw_pd['KW Freq'] = pd.Series([w[1] for w in most_common_lem])
    kw_pd['Bigrams'] = pd.Series([b[0] for b in search_bigrams])
    kw_pd['Bigram Freq'] = pd.Series([b[1] for b in search_bigrams])
    return kw_pd

In [5]:
musowkw = keywords(musow_corpus)
mjikw = keywords(mji_corpus)
differences = musowkw['Bigrams'][0:20].compare(mjikw['Bigrams'][0:20])
differences

,self,other
0,early music,oral history
1,sheet music,history project
2,music collection,american music
3,collection online,audio video
4,music library,sound archive
5,library digital,country music
6,digital score,digital collection
7,digital edition,archive feature
8,song dataset,history collection
9,music information,black music


In [89]:
musowkw['Source'] = 'musow'
mjikw['Source'] = 'mji'
different = pd.concat([musowkw, mjikw]).drop_duplicates(keep=False)
different[21:50]

,Most Common KW,KW Freq,Bigrams,Bigram Freq,Source
21,file,65,collection music,6.0,musow
22,sound,62,sound recording,17.0,musow
23,material,58,musical score,7.0,musow
24,edition,57,digitized manuscript,6.0,musow
25,composer,57,collection digital,6.0,musow
26,information,56,music score,11.0,musow
27,dataset,56,manuscript score,6.0,musow
28,digitized,47,century music,9.0,musow
29,jazz,47,music database,15.0,musow
30,american,46,university library,8.0,musow


In [5]:
#clean and get 35 top words 
string = musow_corpus.raw('')
tokenised = punct_tokenizer.tokenize(string)
clean = [w for w in tokenised if w not in stopwords]
clean_2 = [w for w in clean if w not in custom_stopwords]
#clean_2 = TreebankWordDetokenizer().detokenize(clean)
#text_final = ' '.join(clean_2.split())
#text_final_toks = nltk.word_tokenize(text_final)
freqdist_clean = nltk.FreqDist(clean_2)
#freqdist_final = nltk.FreqDist(text_final_toks)
most_common = freqdist_clean.most_common(35)
most_common_list = []
for t in most_common:
    most_common_list.append(t[0])



In [ ]:
#lemmatized cleaned tokens 
lemmatizer = WordNetLemmatizer()
lemmatized = [lemmatizer.lemmatize(w) for w in clean_2]
freqdist_lem = nltk.FreqDist(lemmatized)
most_common_lem = freqdist_lem.most_common(35)
most_common_lem_list = []
for t in most_common_lem:
    most_common_lem_list.append([t[0], t[1]])
most_common_lem_list

In [35]:
# get ngrams if both are in most common list
bigrams = nltk.bigrams(lemmatized)
freqdist_bg = nltk.FreqDist(bigrams)
search_bigrams = []
for k, v in freqdist_bg.items():
    if k[0] in most_common_lem_list[0]:
        if k[1] in most_common_lem_list[0]:
            if v > 10:
                k = ' '.join(k)
                search_bigrams.append([k, v])


In [39]:
search_bigrams

[]